In [3]:
import os
import time

import openai
import pandas as pd
import requests
from dotenv import load_dotenv
from openai.embeddings_utils import cosine_similarity
from tqdm import tqdm

load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [4]:
title = 'BOOM vs TALON - SEA QUALIFIER FINAL - ESL ONE MALAYSIA 2022 SEA QUALIFIERS Dota 2 Highlights'

TournamentID не коррелирует с датой начала турнира

In [10]:
r = requests.get('https://api.opendota.com/api/leagues')
r.raise_for_status()
all_tournaments = r.json()
all_tournaments.sort(key=lambda dct: -dct['leagueid'])
all_tournaments[:5]

[{'leagueid': 65019,
  'ticket': None,
  'banner': None,
  'tier': 'professional',
  'name': 'BanglaGamer Dota 2 Championship'},
 {'leagueid': 65018,
  'ticket': None,
  'banner': None,
  'tier': 'professional',
  'name': 'GameLadders DOTA 2 League '},
 {'leagueid': 65017,
  'ticket': 'econ/leagues/subscriptions_dreamchallenge',
  'banner': 'econ/leagues/subscriptions_dreamchallenge_ingame',
  'tier': None,
  'name': 'Dota 2 Holiday Dream Challenge'},
 {'leagueid': 65016,
  'ticket': 'econ/leagues/subscriptions_ixdlsealeague',
  'banner': 'econ/leagues/subscriptions_ixdlsealeague_ingame',
  'tier': None,
  'name': 'IXDL Southeast Asian Invite League'},
 {'leagueid': 65015,
  'ticket': 'econ/leagues/subscriptions_slcg',
  'banner': 'econ/leagues/subscriptions_slcg_ingame',
  'tier': None,
  'name': 'Sri Lanka Cyber Games 2013'}]

Teams

In [13]:
r = requests.get('https://api.opendota.com/api/teams')
r.raise_for_status()
all_teams = r.json()
all_teams.sort(key=lambda dct: -dct['last_match_time'])
all_teams[:3]

[{'team_id': 8203841,
  'rating': 1107.38,
  'wins': 10,
  'losses': 3,
  'last_match_time': 1659278486,
  'name': 'DestroyItems',
  'tag': 'DI',
  'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1680367691784130566/110EDF7E5BDD7D305792871B1FC584B52A5794A8/'},
 {'team_id': 8469455,
  'rating': 1182.68,
  'wins': 937,
  'losses': 374,
  'last_match_time': 1659276451,
  'name': 'DEEP RAVAGE',
  'tag': 'DR',
  'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1707410463457648143/58CBD89C985483CDA068FA7A9CFDBA40A81FB245/'},
 {'team_id': 8750317,
  'rating': 1176.45,
  'wins': 88,
  'losses': 56,
  'last_match_time': 1659276451,
  'name': 'Team Sphinx',
  'tag': 'TSX',
  'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1861688906825321280/B904083AF403DC27F8DC18EA96E2406AEB64BFDA/'}]

In [19]:
for t in all_teams:
    if (t['tag'] and t['tag'].upper() in title) or (t['name'] and t['name'].upper() in title):
        print(t)

{'team_id': 8597976, 'rating': 1396.52, 'wins': 81, 'losses': 30, 'last_match_time': 1658903414, 'name': 'Talon', 'tag': 'TLN', 'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1818886756398782872/1DA3D79899C2200A5B0B9DF361BC278D572CE314/'}
{'team_id': 7732977, 'rating': 1473.62, 'wins': 343, 'losses': 206, 'last_match_time': 1658826027, 'name': 'BOOM Esports', 'tag': 'BOOM', 'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/768365068750604173/EF3216F4B02A0DA2327AC3190A369349C7BF3A24/'}
{'team_id': 6120499, 'rating': 1072.77, 'wins': 15, 'losses': 7, 'last_match_time': 1608472089, 'name': 'Flex', 'tag': 'F', 'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/970976016474225549/49E128B88EF90CC3A774AD65C2F180369805F773/'}
{'team_id': 3785359, 'rating': 1152.17, 'wins': 159, 'losses': 134, 'last_match_time': 1581669735, 'name': 'BOOM ID', 'tag': 'BOOM', 'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/763846429146901501/E85EF58254E8BA84ED8A085512ECA0A6499499FE

Последние матчи

In [5]:
r = requests.get('https://api.opendota.com/api/proMatches')
r.raise_for_status()
matches = r.json()

In [13]:
df_matches = pd.DataFrame(matches)
print(df_matches.columns)
df_matches.head()

Index(['match_id', 'duration', 'start_time', 'radiant_team_id', 'radiant_name',
       'dire_team_id', 'dire_name', 'leagueid', 'league_name', 'series_id',
       'series_type', 'radiant_score', 'dire_score', 'radiant_win'],
      dtype='object')


,match_id,duration,start_time,radiant_team_id,radiant_name,dire_team_id,dire_name,leagueid,league_name,series_id,series_type,radiant_score,dire_score,radiant_win
0,6685951110,739,1659260127,8741331,Atomic Esports,8757202,Dream Walker,14426,Victorious Cup,690764,0,19,4,True
1,6685948246,2129,1659259965,8538804,Atlantis,8750069,Gorilla,14323,Asian Tigers 3,690752,1,43,40,False
2,6685920449,2299,1659258706,8607589,EYE-gaming,8746180,NINFAW,14200,Ultras Dota Pro League,690759,1,28,39,False
3,6685919741,2143,1659258645,8765917,Shinigami Gaming,8469455,DEEP RAVAGE,14200,Ultras Dota Pro League,690758,1,27,40,False
4,6685893938,1414,1659257496,8750069,Gorilla,8538804,Atlantis,14323,Asian Tigers 3,690752,1,25,44,False


In [28]:
tournaments = df_matches['league_name'].unique()
print(tournaments)

['Victorious Cup' 'Asian Tigers 3' 'Ultras Dota Pro League '
 'Khaz Modan Cup Series Season 3' 'Asia Battle Arena'
 'Titus Colosseum Cup Season 2'
 'ESL One Malaysia 2022 Qualifiers powered by Intel'
 'TodayPay Dota2 Invitational Season 1' 'Thunderpick Bitcoin Series'
 'European Pro League Season 2'
 'Соревнования по электронным играм «Е-динство»' 'Destiny league']


In [29]:
tournament_embedding = dict()
for t in tournaments:
    embedding = openai.Embedding.create(
        input=t,
        engine='text-similarity-davinci-001'
    )["data"][0]["embedding"]
    tournament_embedding[t] = embedding

In [36]:
title_embedding = openai.Embedding.create(
    input=title.replace(' Dota 2 Highlights', ''),
    engine='text-similarity-davinci-001'
)["data"][0]["embedding"]

In [54]:
similarities = [
    (t_name, cosine_similarity(title_embedding, t_emb))
    for t_name, t_emb in tournament_embedding.items()
]
similarities.sort(key=lambda tpl: -tpl[1])
similarities

[('ESL One Malaysia 2022 Qualifiers powered by Intel', 0.8548940563525276),
 ('TodayPay Dota2 Invitational Season 1', 0.8271779300503733),
 ('Titus Colosseum Cup Season 2', 0.7806075820901524),
 ('Соревнования по электронным играм «Е-динство»', 0.7760129689589339),
 ('Khaz Modan Cup Series Season 3', 0.7730935420368709),
 ('European Pro League Season 2', 0.7496516101333087),
 ('Asia Battle Arena', 0.7305723128544946),
 ('Ultras Dota Pro League ', 0.7278138448099676),
 ('Thunderpick Bitcoin Series', 0.7275276065108286),
 ('Destiny league', 0.6882182843531557),
 ('Asian Tigers 3', 0.6838774460409356),
 ('Victorious Cup', 0.6679672853426503)]

In [41]:
teams = df_matches['radiant_name'] + ' vs ' + df_matches['dire_name']
teams

0      Atomic Esports vs Dream Walker
1                 Atlantis vs Gorilla
2                EYE-gaming vs NINFAW
3     Shinigami Gaming vs DEEP RAVAGE
4                 Gorilla vs Atlantis
                   ...               
95      Sworn Die vs Flawless Goblins
96     Raccoon Gangster vs EYE-gaming
97      Swift Knights vs Dragon Blood
98                       LBZS vs wolf
99     EYE-gaming vs Raccoon Gangster
Length: 100, dtype: object

In [43]:
team_tournament = teams + ' - ' + df_matches['league_name']
team_tournament

0       Atomic Esports vs Dream Walker - Victorious Cup
1                  Atlantis vs Gorilla - Asian Tigers 3
2        EYE-gaming vs NINFAW - Ultras Dota Pro League 
3     Shinigami Gaming vs DEEP RAVAGE - Ultras Dota ...
4                  Gorilla vs Atlantis - Asian Tigers 3
                            ...                        
95       Sworn Die vs Flawless Goblins - Destiny league
96    Raccoon Gangster vs EYE-gaming - Ultras Dota P...
97       Swift Knights vs Dragon Blood - Destiny league
98    LBZS vs wolf - ESL One Malaysia 2022 Qualifier...
99    EYE-gaming vs Raccoon Gangster - Ultras Dota P...
Length: 100, dtype: object

In [50]:
team_tournament_embeddings = dict()
for tt in tqdm(team_tournament):
    embedding = openai.Embedding.create(
        input=tt,
        engine='text-similarity-davinci-001'
    )["data"][0]["embedding"]
    team_tournament_embeddings[tt] = embedding
    time.sleep(1.05)

In [53]:
similarities2 = [
    (tt_name, cosine_similarity(title_embedding, tt_emb))
    for tt_name, tt_emb in team_tournament_embeddings.items()
]
similarities2.sort(key=lambda tpl: -tpl[1])
similarities2[:5]

[('BetBoom Team vs Gaimin Gladiators - ESL One Malaysia 2022 Qualifiers powered by Intel',
  0.9286447179583041),
 ('Gaimin Gladiators vs BetBoom Team - ESL One Malaysia 2022 Qualifiers powered by Intel',
  0.9274933955249376),
 ('BetBoom Team vs Entity - ESL One Malaysia 2022 Qualifiers powered by Intel',
  0.9227010373633419),
 ('Entity vs BetBoom Team - ESL One Malaysia 2022 Qualifiers powered by Intel',
  0.9209996207401326),
 ('LBZS vs Team Aster - ESL One Malaysia 2022 Qualifiers powered by Intel',
  0.9191740334986453)]